In [1]:
# 载入所需库包
import datetime
import collections
import jieba
import re
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import metrics
import pandas as pd

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, DataConversionWarning, FitFailedWarning, NotFittedError
# 忽略所有的警告
warnings.filterwarnings('ignore')

In [3]:
#1.读入 分词后的新闻标题、网址及文本 的 Excel档
print ('1.读入 分词后的新闻标题、网址及文本 的 Excel档' )
print('开始时间：' + str(datetime.datetime.now()))

news_df = pd.read_excel('data分词后.xlsx', header=0)

# 检查训练数据是否合规
categories = {'于是':0, '孤独患者':0, '罗生门':0}
i = 0; err = 0
while i < len(news_df):
    if len(str(news_df.分词后内容[i])) < 1: err = err + 1 # 文本少于10个字视为异常
    categories[news_df.歌名[i]] += 1 # 累计各类文本数量
    i = i + 1

# 打印出 各类训练数量、总数量 及 异常数量
print('')
print(categories) 
print('聚类数据数量：' + str(len(news_df)) + '，异常数据数量：' + str(err) 
      + '，结束时间：' + str(datetime.datetime.now()))

# print(news_df[:5])

news_df['聚类'] = ''
news_df['聚类用内容'] = ''

print('\n结束时间：' + str(datetime.datetime.now()))

1.读入 分词后的新闻标题、网址及文本 的 Excel档
开始时间：2024-12-20 19:34:43.251315

{'于是': 55226, '孤独患者': 56084, '罗生门': 53871}
聚类数据数量：165181，异常数据数量：0，结束时间：2024-12-20 19:34:55.396408

结束时间：2024-12-20 19:34:55.413179


In [4]:
#2.抽取词向量特征
print ('2.抽取词向量特征' )
print('开始时间：' + str(datetime.datetime.now()))

docs = news_df['分词后内容'].tolist()  # 更简洁的列表推导式

# 设置特征数量限制
max_features = 10000  # 根据您的内存调整
vectorizer = TfidfVectorizer(max_df=0.7, max_features=max_features)
tfidf = vectorizer.fit_transform(docs)
 
word = vectorizer.get_feature_names_out()

print('tfidf.shape：' + str(tfidf.shape))
print('len(word)：' + str(len(word)))

weight = tfidf.toarray()       # 将 tf-idf 矩阵抽取出来
#元素w[i][j]表示j词在i类文本的 tf-idf 权重

print('tfidf.shape：' + str(tfidf.shape))
print('len(word)：' + str(len(word)))
print('len(weight)：' + str(len(weight)))

print('\n结束时间：' + str(datetime.datetime.now()))

2.抽取词向量特征
开始时间：2024-12-20 19:34:55.420170
tfidf.shape：(165181, 10000)
len(word)：10000
tfidf.shape：(165181, 10000)
len(word)：10000
len(weight)：165181

结束时间：2024-12-20 19:34:56.918477


In [5]:
# 3.设置聚类个数，为了兼顾可解释性、有效性，可通过不同 k 值进行迭代比较
print ('3.设置聚类个数' )
print('开始时间：' + str(datetime.datetime.now()))

k = 3
kmeans = KMeans(n_clusters=k)
#kmini_model = kmini.fit(weight)
pca = PCA(n_components=1000)
newData = pca.fit_transform(weight)
kmeans.fit(weight)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
n_clusters=k, n_init=100, random_state=None, tol=0.001, verbose=0)
print('前十笔文本聚类结果：')
print(kmeans.labels_[:10])

print('\n结束时间：' + str(datetime.datetime.now()))

3.设置聚类个数
开始时间：2024-12-20 19:34:56.936147
前十笔文本聚类结果：
[2 2 2 2 2 2 2 2 2 2]

结束时间：2024-12-20 19:45:51.453507


In [6]:
# 4.各聚类的文本数量及常用词汇
print ('4.各聚类的文本数量及常用词汇' )
print('开始时间：' + str(datetime.datetime.now()))

cluster = [[], [], []]
for i in range(len(kmeans.labels_)):
    if kmeans.labels_[i]==0:
        cluster[0].append(docs[i])
    if kmeans.labels_[i]==1:
        cluster[1].append(docs[i])
    if kmeans.labels_[i]==2:
        cluster[2].append(docs[i])
    if kmeans.labels_[i]==3:
        cluster[3].append(docs[i])
    news_df.loc[i, '聚类'] = str(kmeans.labels_[i])
    news_df.loc[i, '聚类用内容'] = docs[i]

from collections import Counter

for i in range(3):
    print('cluster[' + str(i) + ']：' + str(len(cluster[i])) + ' 条评论，用词总数 及 词频前20名：')
    clustercontent = ' '.join(cluster[i])
#    print('clustercontent：' + str(len(clustercontent)))
    words = clustercontent.split()
#    print('words：' + str(len(words)))
    
    #取词频前20名的字词(改采 collections 库)
    word_freq = Counter(words)
    print(len(word_freq))
    print(word_freq.most_common(20))  
    print(' ')
    
print('结束时间：' + str(datetime.datetime.now()))

4.各聚类的文本数量及常用词汇
开始时间：2024-12-20 19:45:51.543354
cluster[0]：6245 条评论，用词总数 及 词频前20名：
261
[('加油', 6628), ('兄弟', 104), ('宝宝', 85), ('大笑', 83), ('可怜', 45), ('成功', 45), ('陌生人', 42), ('可爱', 40), ('中考', 26), ('比耶', 22), ('高考', 21), ('憨笑', 20), ('天天开心', 15), ('好好', 15), ('努力', 15), ('考上', 13), ('耍酷', 11), ('越来越', 10), ('爱心', 10), ('上岸', 9)]
 
cluster[1]：3665 条评论，用词总数 及 词频前20名：
2759
[('希望', 3965), ('幸福', 423), ('开心', 304), ('好好', 248), ('永远', 219), ('可怜', 208), ('天天开心', 195), ('下次', 181), ('喜欢', 178), ('大笑', 175), ('大哭', 163), ('生活', 158), ('难过', 145), ('有人', 131), ('快乐', 128), ('身边', 125), ('真的', 121), ('加油', 103), ('明年', 94), ('遇见', 91)]
 
cluster[2]：155271 条评论，用词总数 及 词频前20名：
39644
[('喜欢', 13813), ('幸福', 11274), ('真的', 6556), ('永远', 6524), ('大哭', 4567), ('流泪', 4415), ('世界', 3889), ('朋友', 3757), ('开心', 3581), ('难过', 3532), ('好好', 3409), ('冬天', 3394), ('生活', 3298), ('眼泪', 3121), ('不想', 3095), ('好像', 3089), ('我爱你', 3029), ('回忆', 2978), ('时间', 2909), ('大笑', 2716)]
 
结束时间：2024-12-20 19:46:04.458463

In [7]:
# 5.将聚类后的 新闻 DataFrame 存入 Excel 档
print ('5.将聚类后的 新闻 DataFrame 存入 Excel 档' )
print('开始时间：' + str(datetime.datetime.now()))

news_df.to_excel('data聚类后.xlsx', header=True)

print('\n结束时间：' + str(datetime.datetime.now()))

5.将聚类后的 新闻 DataFrame 存入 Excel 档
开始时间：2024-12-20 19:46:04.477147

结束时间：2024-12-20 19:46:25.985380


In [8]:
# 6.将聚类后的结果作成交叉分析表
print ('6.将聚类后的结果作成交叉分析表' )
print('开始时间：' + str(datetime.datetime.now()))
print(' ')

print(pd.crosstab(news_df['歌名'], news_df['聚类'], margins=True, margins_name='合计'))

print('\n结束时间：' + str(datetime.datetime.now()))

6.将聚类后的结果作成交叉分析表
开始时间：2024-12-20 19:46:25.993139
 
聚类       0     1       2      合计
歌名                              
于是     737  1315   53174   55226
孤独患者  1751  1205   53128   56084
罗生门   3757  1145   48969   53871
合计    6245  3665  155271  165181

结束时间：2024-12-20 19:46:26.089978
